<a href="https://colab.research.google.com/github/WenjunYuAnny/AI_Project_Foreign_Whisper/blob/output-video-with-audio-subtitle/Text_to_speech_with_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from scipy.io.wavfile import write as write_wav

In [2]:
pip install git+https://github.com/suno-ai/bark.git

  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-xg8pysxg
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-xg8pysxg
  Resolved https://github.com/suno-ai/bark.git to commit 773624d26db84278a55aacae9a16d7b25fbccab8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 24.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.7 MB/s eta 0:00:00
  Created wheel for suno-bark: filename=suno_bark-0.0.1a0-py3-none-any.whl size=2567

In [3]:
from bark.generation import (
    preload_models,
)
from bark import generate_audio, SAMPLE_RATE
import numpy as np
import librosa

In [4]:
import os

In [5]:
os.environ["SUNO_USE_SMALL_MODELS"] = "1"

In [6]:
preload_models()

text_2.pt:   0%|          | 0.00/5.35G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

coarse_2.pt:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

fine_2.pt:   0%|          | 0.00/3.74G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:00<00:00, 115MB/s]


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
def srt_to_subtitles(srt_file_path):
    with open(srt_file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    subtitles = []
    current_subtitle = {}

    for line in lines:
        line = line.strip()
        if line.isdigit():
            if current_subtitle:
                subtitles.append(current_subtitle)
            current_subtitle = {}
        elif '-->' in line:
            start, end = map(float, line.split(" --> "))
            current_subtitle["start"] = start
            current_subtitle["end"] = end
        elif line:
            current_subtitle["text"] = line
    if current_subtitle:
        subtitles.append(current_subtitle)
    return subtitles

In [9]:
def text_to_speech(subtitles, output_file_path):
  SPEAKER = "v2/fr_speaker_0"
  pieces = []
  for i, subtitle in enumerate(subtitles):
      print("start: " +  subtitle["text"])
      text = subtitle["text"]
      audio_array = generate_audio(text, history_prompt=SPEAKER)
      audio_array = audio_array.squeeze()
      speed_factor = (subtitle["end"] - subtitle["start"]) / len(audio_array) * SAMPLE_RATE
      print("video_original_length ", subtitle["end"] - subtitle["start"])
      print("output length ", len(audio_array) / SAMPLE_RATE)
      audio_array = librosa.effects.time_stretch(audio_array, rate=(1/speed_factor))
      pieces.append(audio_array)
      if i < len(subtitles) - 1:
          pause = subtitles[i+1]["start"] - subtitles[i]["end"]
          if pause > 0:
            pause_audio = np.zeros(int(pause * SAMPLE_RATE))
            pieces.append(pause_audio)
  concatenated_audio = np.concatenate(pieces)
  write_wav(output_file_path, rate=SAMPLE_RATE, data=concatenated_audio)

In [10]:
srt_file_path = "/content/drive/MyDrive/2023 Fall Artificial Intelligence/Foreign Whisper/translated_srt/Attorney General Merrick Garland The 60 Minutes Interview.srt"

In [11]:
filename = os.path.splitext(os.path.basename(srt_file_path))[0] + '.wav'

In [12]:
output_file_dir = "/content/drive/MyDrive/2023 Fall Artificial Intelligence/Foreign Whisper/translated_audio_time"
output_file_path = os.path.join(output_file_dir, filename)

In [13]:
subtitles = srt_to_subtitles(srt_file_path)

In [14]:
print(subtitles)

[{'start': 0.0, 'end': 8.0, 'text': 'Depuis Watergate, un procureur général n’a pas été au centre d’une telle tempête d’incendie.'}, {'start': 8.0, 'end': 16.0, 'text': "Le ministère de la Justice de Merrick Garland poursuiv l'ancien président Trump et le fils du président Biden."}, {'start': 16.0, 'end': 24.0, 'text': 'Au milieu de cette affaire se trouve cet ancien procureur et juge de renommée de 70 ans, ayant une longue histoire de modéré.'}, {'start': 24.0, 'end': 36.0, 'text': "Nous avons rencontré Garland vendredi à Washington et il nous a dit qu'il consacrait sa vie à l'état de droit en raison de la lutte de sa famille pour échapper à l'Holocauste."}, {'start': 36.0, 'end': 41.0, 'text': "Il est maintenant responsable des poursuites qui façonneront l'avenir de la nation."}, {'start': 41.0, 'end': 54.0, 'text': "Dans une rare entrevue, le procureur général nous a dit que dans les procès de Trump et de Biden qui s'annoncent, ses procureurs poursuivront la justice sans crainte et 

In [15]:
text_to_speech(subtitles, output_file_path)

start: Depuis Watergate, un procureur général n’a pas été au centre d’une telle tempête d’incendie.


100%|██████████| 16/16 [00:10<00:00,  1.54it/s]


video_original_length  8.0
output length  6.32
start: Le ministère de la Justice de Merrick Garland poursuiv l'ancien président Trump et le fils du président Biden.


100%|██████████| 17/17 [00:10<00:00,  1.61it/s]


video_original_length  8.0
output length  6.506666666666667
start: Au milieu de cette affaire se trouve cet ancien procureur et juge de renommée de 70 ans, ayant une longue histoire de modéré.


...

video_original_length  9.0
output length  8.026666666666667
start: Je crois que c'est le meilleur que tout fonctionnaire puisse espérer, que nous avons fait de notre mieux, que nous transmettions un ministère de la Justice qui continue de poursuivre la primauté du droit et de la protéger.


100%|██████████| 35/35 [00:20<00:00,  1.71it/s]


video_original_length  13.0
output length  13.84
start: Et c'est la même chose que chaque génération doit espérer, que nous pouvons transmettre notre démocratie en bon état de marche à la prochaine génération qui prend le flambeau et qui est responsable, une fois terminée, de poursuivre ce travail.


100%|██████████| 35/35 [00:20<00:00,  1.68it/s]


video_original_length  19.0
output length  13.96
